# Creating the web scraper

In [2]:
!pip install bs4

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import sklearn.metrics as sm


import re

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import csv
import requests
import json
from pandas.io.json import json_normalize
from urllib.request import urlopen
from bs4 import BeautifulSoup


from neo4j import GraphDatabase 
from py2neo import Graph,Node,Relationship

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=c0cf1aeb918257800854841a0afc9a77832484069f64fbbdccf5c011cbbbc152
  Stored in directory: c:\users\marti\appdata\local\pip\cache\wheels\25\42\45\b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


# Getting the dataset

In [66]:
data = pd.read_csv('../data/music-subset.csv')

In [67]:
refined_data = data.loc[(data["rank"] == 1)]

In [68]:
# new_index = [1,2,3,4,5,6,7,8,9,10,11,12,133,14,15,16,17,18,19,20]

In [69]:
new_data = refined_data.reset_index(drop = True)

In [70]:
refined_data.shape

(6, 22)

In [71]:
index_names = new_data[new_data['trend'] == 'SAME_POSITION' ].index

In [72]:
new_data.drop(index_names, inplace=True)

In [73]:
new_data.shape

(3, 22)

In [74]:
new_data

,index,title,rank,date,artist,url,region,chart,trend,streams,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
1,201,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0,...,7.0,-7.028,1.0,0.2820,0.1650,0.000003,0.134,0.535,186.054,230453.0
2,402,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0,...,10.0,-3.194,1.0,0.0458,0.1860,0.000405,0.121,0.157,97.019,161200.0
3,603,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0,...,6.0,-4.582,0.0,0.0845,0.0154,0.000000,0.235,0.831,90.018,190813.0


In [77]:
contain_values = data[data.artist.str.contains('The Weeknd')]
print (contain_values)

     index                 title  rank        date  \
28      28               Starboy    29  2017-01-01   
75      75      I Feel It Coming    76  2017-01-01   
201    201               Starboy     1  2017-01-01   
207    207      I Feel It Coming     7  2017-01-01   
266    266         Party Monster    66  2017-01-01   
307    307               Rockin’   107  2017-01-01   
331    331    Can't Feel My Face   130  2017-01-01   
333    333             Sidewalks   132  2017-01-01   
368    368  Starboy - Kygo Remix   167  2017-01-01   
388    388        Six Feet Under   187  2017-01-01   
394    394             The Hills   193  2017-01-01   
408    408               Starboy     7  2017-01-01   
449    449      I Feel It Coming    47  2017-01-01   
560    560         Party Monster   158  2017-01-01   
605    605               Starboy     3  2017-01-01   
621    621      I Feel It Coming    19  2017-01-01   
725    725         Party Monster   123  2017-01-01   
819    819               Sta

In [76]:
#200403  I Feel It Coming
#209094   Save Your Tears
# 1 Starboy

## Reducing the amount of data

In [13]:
new_data.head()

,index,title,rank,date,artist,url,region,chart,trend,streams,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
1,201,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0,...,7.0,-7.028,1.0,0.2820,0.1650,0.000003,0.134,0.535,186.054,230453.0
2,402,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0,...,10.0,-3.194,1.0,0.0458,0.1860,0.000405,0.121,0.157,97.019,161200.0
3,603,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0,...,6.0,-4.582,0.0,0.0845,0.0154,0.000000,0.235,0.831,90.018,190813.0


In [14]:
#new_data = new_data.drop(new_data[new_data.index > 20].index)                          # Using logical condition
   

In [15]:
new_data.shape

(3, 22)

In [16]:
new_data.iloc[140087:140100,:]

,index,title,rank,date,artist,url,region,chart,trend,streams,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms


In [17]:
new_data

,index,title,rank,date,artist,url,region,chart,trend,streams,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
1,201,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0,...,7.0,-7.028,1.0,0.2820,0.1650,0.000003,0.134,0.535,186.054,230453.0
2,402,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0,...,10.0,-3.194,1.0,0.0458,0.1860,0.000405,0.121,0.157,97.019,161200.0
3,603,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0,...,6.0,-4.582,0.0,0.0845,0.0154,0.000000,0.235,0.831,90.018,190813.0


# Import dataset

In [18]:
data = pd.read_csv('../data/music-subset.csv', index_col=0)

In [19]:
data.head(40)

,title,rank,date,artist,url,region,chart,trend,streams,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
index,,,,,,,,,,,,,,,,,,,,,
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0,0.852,...,8.0,-2.921,0.0,0.0776,0.18700,0.000030,0.1590,0.9070,102.034,195840.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0,0.663,...,11.0,-4.070,0.0,0.2260,0.00431,0.000017,0.1010,0.5330,99.935,259196.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0,0.761,...,4.0,-3.073,0.0,0.0502,0.40000,0.000000,0.1760,0.7100,93.974,222560.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0,0.508,...,0.0,-4.361,1.0,0.3260,0.55100,0.000003,0.1260,0.5550,180.044,205600.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0,0.899,...,6.0,-4.228,0.0,0.2920,0.07600,0.000000,0.0631,0.8730,88.007,234320.0
5,Traicionera,6,2017-01-01,Sebastian Yatra,https://open.spotify.com/track/5J1c3M4EldCfNxX...,Argentina,top200,MOVE_DOWN,151140.0,0.776,...,11.0,-4.933,1.0,0.0638,0.14200,0.000000,0.2190,0.6610,91.012,228467.0
6,Cuando Se Pone a Bailar,7,2017-01-01,Rombai,https://open.spotify.com/track/1MpKZi1zTXpERKw...,Argentina,top200,MOVE_DOWN,148369.0,0.588,...,11.0,-7.169,0.0,0.1730,0.08510,0.000027,0.0840,0.9370,205.643,195274.0
7,Otra vez (feat. J Balvin),8,2017-01-01,Zion & Lennox,https://open.spotify.com/track/3QwBODjSEzelZyV...,Argentina,top200,MOVE_DOWN,143004.0,0.832,...,10.0,-5.429,1.0,0.1000,0.05590,0.000486,0.4400,0.7040,96.016,209453.0
8,La Bicicleta,9,2017-01-01,"Carlos Vives, Shakira",https://open.spotify.com/track/0sXvAOmXgjR2QUq...,Argentina,top200,MOVE_UP,126389.0,0.736,...,0.0,-2.147,1.0,0.1290,0.19800,0.000002,0.3360,0.9530,179.935,227707.0


# Extracting artist and track name

In [46]:
track_nr = 28

In [47]:
import re

def get_track_name(nr):
    

    track_string = str(data.title[nr])
    
    x = track_string.find(" (")
    y = track_string.find(" -")


    
    if(int(x) < 0):
        track_string = track_string
    if(int(x) > 0):
        track_string = track_string[:x]
    if (int(y) > 0):
        track_string = track_string[:y]
    
    return track_string



def get_track_artist(nr):
    
    artist_string = str(data.artist[nr])

    y = artist_string.find(", ")
    x = artist_string.find(" (")
    
    if(int(x) > 0):
        artist_string = artist_string.replace("' ", "")

    if(int(y) > 0):
        #artist_string = y
        artist_string = artist_string[:y]     
        
    return artist_string

In [48]:
get_track_name(track_nr)

'Starboy'

In [49]:
get_track_artist(track_nr)

'The Weeknd'

# Getting URL

In [50]:
import re

def get_url(track_nr):
    
    track = str(get_track_name(track_nr))
    artist = str(get_track_artist(track_nr))
    
    track = re.sub('[^a-zA-Z0-9 \n\.]', '', track)
    artist = re.sub('[^a-zA-Z0-9 \n\.]', '', artist)
    
    
    if (int(artist.find(" ")) > 0):
        artist = artist.replace(" ", "-")
        
    if (int(track.find(" ")) > 0):
        track = track.replace(" ", "-")
        
    
    
    url_string = str('https://genius.com/' + artist + '-' + track + '-lyrics')
    
    return str(url_string)
    

In [51]:
get_url(track_nr)

'https://genius.com/The-Weeknd-Starboy-lyrics'

# Scraping credits

In [52]:
def extract_credits(URL):
    
    # "https://genius.com/Shakia-chantaje-lyrics" <- url to check for raised exception
    
    r = requests.get(URL)
    
    if (int(r.status_code) == 200):


        # Parsing the HTML
        soup = BeautifulSoup(r.content, 'html.parser')
        s = soup.find('div', class_='SongInfo__Columns-nekw6x-2 lgBflw')


        featur = []
        writer = []
        label = []
        producer = []

        for line in s:
            div = line.find('div').text
            an = line.find_all('a')


            if 'Featuring' in div: 
                extract = [x.text for x in an]
                for x in extract:
                    featur.append(x)



            if 'Written By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    writer.append(x)



            if 'Label' in div:
                extract = [x.text for x in an]
                for x in extract:
                    label.append(x)

            if 'Produced By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    producer.append(x)

    else:
        raise Exception("Sorry, no numbers below zero") 

    collection = {"Feature":featur, "Writer":writer, "Label":label, "Producer":producer}        
        
        
    return collection

In [53]:
#extLis = extract_credits(get_url(218900))
#extLis = extract_credits(get_url(200403))
#extLis = extract_credits(get_url(98))
extLis = extract_credits(get_url(track_nr))

# Dict to csv

In [54]:
credit_list = pd.DataFrame.from_dict(extLis, orient='index').T

### Path to neo4j import folder
Nicholas Stationær: C:\Users\Nmtur\.Neo4jDesktop\relate-data\dbmss\dbms-20b18527-223b-4ada-8e62-429ad982fc9b\import\collaborators.csv\
Martins bærbare: C:\Users\marti\.Neo4jDesktop\relate-data\dbmss\dbms-a7f58857-34e9-48ad-b898-3dc31e11d74c\import\collaborators.csv

In [55]:
df = pd.DataFrame.from_dict(credit_list) 
df.to_csv (r'C:\Users\marti\.Neo4jDesktop\relate-data\dbmss\dbms-a7f58857-34e9-48ad-b898-3dc31e11d74c\import\collaborators.csv', index = False, header=True)

In [56]:
df

,Feature,Writer,Label,Producer
0,Daft Punk,Guy-Manuel de Homem-Christo,XO Records,Daft Punk
1,None,Thomas Bangalter,Universal Music Group,Cirkut
2,None,DaHeala,Republic Records,Doc McKinney
3,None,The Weeknd,None,The Weeknd
4,None,Doc McKinney,None,None
5,None,Cirkut,None,None


***

# Neo4j

In [57]:
# Create connection to database

class neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
                print("Failed to create the driver: ", e)
                
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed!", e)
        finally:
            if session is not None:
                session.close()
        return response
    
    def close(self):
        if self.__driver is not None:
            self.__driver.close()

In [58]:
# Connecting to database
conn = neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="test1234")


In [59]:
conn.query('''MATCH (n) RETURN n;''')

[<Record n=<Node id=0 labels=frozenset({'Person'}) properties={'name': 'Daft Punk'}>>,
 <Record n=<Node id=2 labels=frozenset({'Person'}) properties={'name': 'The Weeknd'}>>,
 <Record n=<Node id=3 labels=frozenset({'Person'}) properties={'name': 'Rico The Wizard'}>>,
 <Record n=<Node id=4 labels=frozenset({'Person'}) properties={'name': 'Guy-Manuel de Homem-Christo'}>>,
 <Record n=<Node id=5 labels=frozenset({'Person'}) properties={'name': 'Thomas Bangalter'}>>,
 <Record n=<Node id=6 labels=frozenset({'Person'}) properties={'name': 'Doc McKinney'}>>,
 <Record n=<Node id=7 labels=frozenset({'Person'}) properties={'name': 'Cirkut'}>>,
 <Record n=<Node id=8 labels=frozenset({'Label'}) properties={'name': 'XO Records'}>>,
 <Record n=<Node id=9 labels=frozenset({'Label'}) properties={'name': 'Universal Music Group'}>>,
 <Record n=<Node id=10 labels=frozenset({'Label'}) properties={'name': 'Republic Records'}>>,
 <Record n=<Node id=12 labels=frozenset({'Song'}) properties={'name': 'I Feel It

In [86]:
artist = get_track_artist(track_nr)
track = get_track_name(track_nr)
track

'Starboy'

In [87]:
artist

'The Weeknd'

In [88]:
data.title[28]

'Starboy'

### Query to insert into neo4j

In [95]:
def run_querys(track_nr):
    
    artist = get_track_artist(track_nr)
    track = get_track_name(track_nr)
    
    conn.query('''CALL apoc.load.csv('collaborators.csv')
    YIELD lineNo, map, list
    
    WHERE map.Feature IS NOT NULL
    MERGE (f:Person {name:map.Feature})
    MERGE (a:Person {name:"'''+ artist + '''"})
    MERGE (w:Person {name:map.Writer})
    MERGE (p:Person {name:map.Producer})
    MERGE (l:Label {name:map.Label})
    MERGE (s:Song {name:"'''+ track +'''"})
    
    
    MERGE (f)-[:COLLABORATED]-(a)
    MERGE (f)-[:FEATURED]->(s)
    MERGE (a)-[:SANG]->(s)
    MERGE (w)-[:WROTE]->(s)
    MERGE (l)-[:RELEASED]->(s)
    MERGE (p)-[:PRODUCED]->(s);
    ''')
    
    conn.query('''MATCH (f:Person) WHERE (f.name = "") detach delete f;''')
    
    conn.query('''MATCH (l:Label) WHERE (l.name = "") detach delete l;''')
    
    conn.close()

In [97]:
run_querys(track_nr)

In [111]:
for x in data.index:
    print(str(get_track_artist(x)) + ' - ' + str(get_track_name(x)))


Shakira - Chantaje
Ricky Martin - Vente Pa' Ca
CNCO - Reggaetón Lento
J Balvin - Safari
Daddy Yankee - Shaky Shaky
Sebastian Yatra - Traicionera
Rombai - Cuando Se Pone a Bailar
Zion & Lennox - Otra vez
Carlos Vives - La Bicicleta
Ozuna - Dile Que Tu Me Quieres
Chino & Nacho - Andas En Mi Cabeza
Thalia - Desde Esa Noche
Maluma - Borro Cassette
Charly Black - Gyal You A Party Animal
Piso 21 - Me llamas
Carlos Vives - La Bicicleta
Enrique Iglesias - DUELE EL CORAZON
DJ Snake - Let Me Love You
Mano Arriba - La Noche No Es para Dormir
Wisin - Vacaciones
Clean Bandit - Rockabye
Chano - Carnavalintro
IAmChino - Ay Mi Dios
Drake - One Dance
Calvin Harris - My Way
Major Lazer - Cold Water
Gente De Zona - La Gozadera
Mano Arriba - Llamame Mas Temprano
The Weeknd - Starboy
Cosculluela - Manicomio
Maluma - Cuatro Babys
Maroon 5 - Don't Wanna Know
Maluma - Sin Contrato
Bruno Mars - 24K Magic
Justin Timberlake - CAN'T STOP THE FEELING!
Juanes - Fuego
J Balvin - Bobo
Martin Garrix - In the Name of L

In [64]:
df

,Feature,Writer,Label,Producer
0,Daft Punk,Guy-Manuel de Homem-Christo,XO Records,Daft Punk
1,None,Thomas Bangalter,Universal Music Group,Cirkut
2,None,DaHeala,Republic Records,Doc McKinney
3,None,The Weeknd,None,The Weeknd
4,None,Doc McKinney,None,None
5,None,Cirkut,None,None


### Getting the Shit

In [2]:
track_nr = 449
# For scraping
extLis = extract_credits(get_url(track_nr))
# Get credit
credit_list = pd.DataFrame.from_dict(extLis, orient='index').T
# Running neo4j query
run_querys(track_nr)

NameError: name 'extract_credits' is not defined